### by Andy Jiménez

# Get prices in Real Time - InvertirOnline

In [0]:
import requests, json, pandas, datetime, bs4, numpy, re, json, chart_studio
from bs4 import BeautifulSoup
#import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
chart_studio.tools.set_credentials_file(username='Andy_py', api_key='tqSN9V8SsQeyDAjWDkFN')

## Argentinian Bonds

We can get the real-time prices from the chart of the bond and replicate the curve

In [0]:
#INTRADIARIO POST REQUEST

# BONO
url = 'https://www.invertironline.com/titulo/cotizacion/BCBA/AY24/BONOS-NACION-ARGENTINA-USD-8.75--2024'

headers={
    'Referer': 'https://www.invertirhttps://www.invertironline.com/titulo/cotizacion/BCBA/AY24/BONOS-NACION-ARGENTINA-USD-8.75--2024/graficador',
    'Cookie': '_ga=GA1.2.1419134137.1529598737; i18n.langtag=es-AR; utm_path=ID_origen=99&utm_source=Newsletter&utm_medium=email&utm_campaign=NL_IOL_Research_AperturaDeMercado_CA_30-Jul-19&embtrk=7i7,-R-21368403-R-,a8bi-R-ac7,n9; isMobile=0; _hjid=5d2124d6-1d1f-4432-b663-01fdcfc5448e; _gcl_au=1.1.1127974570.1569418667; _fbp=fb.1.1569520643782.1145300030; uid=631261; _gid=GA1.2.904748864.1570629820; __sidglobal=libjy2todwnixmpbhcrwxjsz; isLogged=1; _dc_gtm_UA-189938-1=1',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,es;q=0.9,pt;q=0.8',
    'Connection': 'keep-alive',
    'Host': 'www.invertironline.com',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Mobile Safari/537.36'
}

r = requests.post(url, headers=headers)
r_ = BeautifulSoup(r.content,'html.parser')

#TABLE HEADERS
table_header = r_.find('table', id='tablaOpsIntrad').find('thead')

t_headers=[]
for tr in table_header.find_all('tr'):
    row_headers = [td.text for td in tr.find_all('td')]
    t_headers.append(row_headers)
    
t_headers=t_headers[1:][0] #List of lists. Taking the first item

# TABLA INTRADIARIA
table = r_.find('table', id='tablaOpsIntrad').find('tbody')

data=[]
for tr in table.find_all('tr'):
    row_text = [td.text for td in tr.find_all('td')]
    data.append(row_text)

# CONVIRTIENDO A DATAFRAME
df=pandas.DataFrame(data, columns=t_headers,index=None)

# CONFIG PANDAS DECIMALES
# pandas.set_option('display.float_format', lambda x: '%.2f' % x) # Convierte números enteros a decimales

# HORA
df['Hora'] = pandas.to_datetime(df['Hora'])
# df.set_index('Hora',inplace=True)
# PRECIO
df['Precio'] = df['Precio'].str.replace(r'\D+', '').astype('int64')
df.loc[:,'Precio'] /= 1000 # precio en miles
# MONTO
df['Monto'] = df['Monto'].str.replace(r'$', '').str.replace(r',', '.')
# VOLUMEN
df['Volumen'] = df['Volumen'].str.replace(r'\D+', '').astype('int64')


In [0]:
# GRÁFICO INTRADIARIO
fig = go.Figure([go.Scatter(x=df['Hora'], y=df['Precio'])])
py.iplot(fig)

## Get prices when the market is closed 

From the chart online, we can get the prices and the operations volume

In [0]:
#INTRADIARIO GET REQUEST

# BONO
url = 'https://www.invertironline.com/Titulo/GraficoIntradiario?idTitulo=79217&idTipo=3&idMercado=1'

headers={
    'Referer': 'https://www.invertironline.com/titulo/cotizacion/BCBA/AY24/BONOS-NACION-ARGENTINA-USD-8.75--2024/',
    'Cookie': 'i18n.langtag=es-AR; isMobile=0; _ga=GA1.2.500534755.1569450835; _gid=GA1.2.815734302.1569450835; _gcl_au=1.1.36899437.1569450835; _hjid=812ca257-432f-4c29-9c9e-861419d6c1c6; uid=490908; __sidglobal=tneqg1a2pxysdw4wnroxur05; isLogged=1; _dc_gtm_UA-189938-1=1',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,es;q=0.9,pt;q=0.8',
    'Connection': 'keep-alive',
    'Host': 'www.invertironline.com',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Mobile Safari/537.36'
}

r = requests.get(url, headers=headers)
# r.content
data=json.loads(r.text)

In [0]:
hora,precio,volumen=[],[],[]

for i in data:
    h = datetime.datetime.utcfromtimestamp(int(i['FechaHora'])).strftime('%H:%M')
    hora.append(h)
    precio.append(float(i['Ultima']))
    volumen.append(int(i['CantidadNominal']))
    
pretable = {}
pretable["Precio"],pretable["Volumen"] = precio,volumen
table = pandas.DataFrame(pretable, columns=["Precio","Volumen"],index=hora)
print("Ultimas operaciones\n")
table.tail(20)

Ultimas operaciones



,Precio,Volumen
16:54,2425.0,14403
16:54,2424.0,300
16:54,2422.0,1000
16:55,2425.0,250
16:55,2428.0,25734
16:55,2428.0,6000
16:55,2421.0,26000
16:57,2382.6,2631580
16:58,2394.0,2631580
17:00,2394.0,2631580


In [0]:
fig = go.Figure([go.Scatter(x=df['Hora'], y=df['Precio'])])
py.iplot(fig)